In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
import skimage.io
%matplotlib inline


In [3]:
tf.reset_default_graph()
mnist   = input_data.read_data_sets('data/', one_hot=True)
TRimg   = mnist.train.images
TRlable = mnist.train.labels
TSimg   = mnist.test.images
TSlabel = mnist.test.labels

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
n_input = 784
n_output = 10

d = "/gpu:0"
with tf.device(d):
    weights = {
        'wc1': tf.Variable(tf.random_normal([3,3,1,64], stddev=0.1)),
        'wd1': tf.Variable(tf.random_normal([14*14*64, n_output], stddev=0.1))
    }
    biases = {
        'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
        'bd1': tf.Variable(tf.random_normal([n_output], stddev=0.1))
    }
    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_output])


In [5]:
def conv_simple(_input, _w, _b):
    _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
    _conv1   = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1,1,1,1], padding='SAME')
    _conv2   = tf.nn.bias_add(_conv1, _b['bc1'])
    _conv3   = tf.nn.sigmoid(_conv2)
    _pool1   = tf.nn.max_pool(_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    _dense   = tf.reshape(_pool1, [-1, _w['wd1'].get_shape().as_list()[0]])
    _out     = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
    
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'conv2' : _conv2, 'conv3' : _conv3, 'pool1': _pool1, 'dense': _dense, 'out': _out
    }
    return out
                                    
    

In [6]:
learning_rate   = 0.001
training_epochs = 10
batch_size      = 100
display_step    = 1

save_step = 1;
savedir = "nets/"
saver = tf.train.Saver(max_to_keep=10)

for d in ['/gpu:0', '/gpu:1', '/gpu:2']:
    with tf.device(d):
        pred = conv_simple(x, weights, biases)['out']
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))#
        optm = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
        corr = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accr = tf.reduce_mean(tf.cast(corr, tf.float32))
        init = tf.global_variables_initializer()


In [ ]:

with tf.device('/gpu:0'):  
    sess = tf.Session()
    sess.run(init)

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_interation = int(mnist.train.num_examples/batch_size)
        for i in range(total_interation):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_interation

        if epoch % display_step ==0:
            print("Epoch: %03d/%03d cost: %.4f" %(epoch, training_epochs, avg_cost))
            train_acc = sess.run(accr, feed_dict={x:batch_xs, y:batch_ys})
            test_acc = sess.run(accr, feed_dict={x:TSimg, y:TSlabel})
            print("Training acc: %.3f, Test acc: %.3f" %(train_acc, test_acc))

        if epoch % save_step == 0:
            saver.save(sess, "nets/cnn_mnist_simple.ckpt-" + str(epoch))


In [ ]:
epoch = 0
saver.restore(sess, "nets/cnn_mnist_simple.ckpt-" + str(epoch))
print ("NETWORK RESTORED")

In [ ]:
filter1=weights['wc1'][:,:,0,0]
fout=sess.run(filter1)
plt.imshow(fout, interpolation='none', cmap=plt.get_cmap('gray'))
fout

In [ ]:
x1=TRimg[10,:]
imgin=sess.run(conv_simple(x1, weights, biases)['input_r'][0,:,:,0])
imgconv1=sess.run(conv_simple(x1, weights,biases)['conv1'][0,:,:,2])
imgconv2=sess.run(conv_simple(x1, weights,biases)['conv2'][0,:,:,2])
imgconv3=sess.run(conv_simple(x1, weights,biases)['conv3'][0,:,:,2])
imgpool=sess.run(conv_simple(x1, weights, biases)['pool1'][0,:,:,0])

plt.figure(1)
plt.matshow(imgin, cmap=plt.get_cmap('gray'))
plt.title("input image")
plt.colorbar()
plt.figure(2)
plt.matshow(imgconv1, interpolation='none', cmap=plt.get_cmap('gray'))
plt.title("convolution")
plt.figure(3)
plt.matshow(imgconv2, interpolation='none', cmap=plt.get_cmap('gray'))
plt.title("bias add")
plt.figure(4)
plt.matshow(imgconv3, interpolation='none', cmap=plt.get_cmap('gray'))
plt.title("sigmoid")
plt.figure(5)
plt.matshow(imgpool, cmap=plt.get_cmap('gray'))
plt.title("max pooling")

In [ ]:
epoch = 9
saver.restore(sess, "nets/cnn_mnist_simple.ckpt-" + str(epoch))
print ("NETWORK RESTORED")

In [ ]:
filter1=weights['wc1'][:,:,0,0]
fout=sess.run(filter1)
plt.imshow(fout, interpolation='none', cmap=plt.get_cmap('gray'))
fout

In [ ]:
x1=TRimg[10,:]
imgin=sess.run(conv_simple(x1, weights, biases)['input_r'][0,:,:,0])
imgconv1=sess.run(conv_simple(x1, weights,biases)['conv1'][0,:,:,2])
imgconv2=sess.run(conv_simple(x1, weights,biases)['conv2'][0,:,:,2])
imgconv3=sess.run(conv_simple(x1, weights,biases)['conv3'][0,:,:,2])
imgpool=sess.run(conv_simple(x1, weights, biases)['pool1'][0,:,:,0])

plt.figure(1)
plt.matshow(imgin, cmap=plt.get_cmap('gray'))
plt.title("input image")
plt.colorbar()
plt.figure(2)
plt.matshow(imgconv1, interpolation='none', cmap=plt.get_cmap('gray'))
plt.title("convolution")
plt.figure(3)
plt.matshow(imgconv2, interpolation='none', cmap=plt.get_cmap('gray'))
plt.title("bias add")
plt.figure(4)
plt.matshow(imgconv3, interpolation='none', cmap=plt.get_cmap('gray'))
plt.title("sigmoid")
plt.figure(5)
plt.matshow(imgpool, cmap=plt.get_cmap('gray'))
plt.title("max pooling")